In [1]:
from flask import Flask, request
from twilio.rest import Client as TwilioClient
from twilio.twiml.messaging_response import MessagingResponse
import threading
import os
from dotenv import load_dotenv
from pyngrok import ngrok
import requests
from datetime import datetime


In [2]:
# Run event_recommender
%run event_recommender.ipynb

# Load environment variables
load_dotenv()

# Initialize clients
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

First event keys: ['Event name', 'Date', 'Start Time', 'Artists', 'Venue', 'Number of guests attending', 'Event URL']
First event data: {'Event name': "NT's Loft 10th Birthday: Free Rooftop Party", 'Date': '2025-09-06T00:00:00+00:00', 'Start Time': '2025-09-06T12:00:00+00:00', 'Artists': "Alec Falconer, Alexander Nut, Ariane V, Charlie Dark, Dean Bryce, Ella Knight, Finn, Jive Talk, Lulah Francs, Madelic, Sam Bangura, Scarlett O'Malley, Silverlining, Swoose, ANLON, Danny Vito, Hannah Lamb, Sharon Khan", 'Venue': 'Night Tales Loft', 'Number of guests attending': 4701, 'Event URL': '/events/2223095'}
• NT's Loft 10th Birthday: Free Rooftop Party (2025-09-06T00:00:00+00:00) @ Night Tales Loft - Alec Falconer, Alexander Nut, Ariane V, Charlie Da (4701 going) | https://ra.co/events/2223095
• Ancestorz - The Free Afro house Day party (2025-09-07T00:00:00+00:00) @ Pop Brixton (2062 going) | https://ra.co/events/2218870
• Jungle Splash (2025-09-05T00:00:00+00:00) @ Brixton Jamm - Potential Bad

In [3]:
# Twilio credentials
TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
twilio_client = TwilioClient(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

In [4]:

# User context storage (in production, use a database)
user_contexts = {}

# Create Flask app
app = Flask(__name__)

def get_user_context(phone_number):
    """Manage user conversation context"""
    if phone_number not in user_contexts:
        user_contexts[phone_number] = {
            'conversation_history': [],
            'preferred_genres': [],
            'first_interaction': True,
            'last_active': datetime.now()
        }
    return user_contexts[phone_number]

def extract_genres_from_message(message):
    """Extract music genres mentioned in the message"""
    genres = []
    genre_keywords = {
        'techno': ['techno', 'tech'],
        'house': ['house', 'deep house', 'tech house'],
        'drum and bass': ['dnb', 'drum and bass', 'jungle'],
        'dubstep': ['dubstep', 'bass'],
        'trance': ['trance', 'progressive'],
        'ambient': ['ambient', 'chill'],
        'breakbeat': ['breaks', 'breakbeat']
    }

    message_lower = message.lower()
    for genre, keywords in genre_keywords.items():
        if any(keyword in message_lower for keyword in keywords):
            genres.append(genre)

    return genres

@app.route('/webhook/whatsapp', methods=['POST'])
def handle_whatsapp():
    """Handle incoming WhatsApp messages"""
    from_number = request.form['From']  # Will be like 'whatsapp:+447123456789'
    message_body = request.form['Body'].strip()

    print(f"💬 WhatsApp from {from_number}: {message_body}")

    # Get user context (clean the whatsapp: prefix for storage)
    clean_number = from_number.replace('whatsapp:', '')
    context = get_user_context(clean_number)

    # Update conversation history
    context['conversation_history'].append(f"User: {message_body}")
    context['last_active'] = datetime.now()

    # Extract and store genre preferences
    mentioned_genres = extract_genres_from_message(message_body)
    for genre in mentioned_genres:
        if genre not in context['preferred_genres']:
            context['preferred_genres'].append(genre)

    try:
        # Get events from your database
        events = get_upcoming_events()  # Your existing function

        if not events:
            response_text = "Hey! I don't see any events right now. Check back later! 🎵"
        else:
            # Format events for LLM
            events_text = format_events_for_llm(events)

            # Generate recommendation
            response_text = generate_recommendations(message_body, events_text, context)

        # Update conversation history
        context['conversation_history'].append(f"Assistant: {response_text}")
        context['first_interaction'] = False

        # Keep history manageable
        if len(context['conversation_history']) > 10:
            context['conversation_history'] = context['conversation_history'][-10:]

    except Exception as e:
        print(f"❌ Error processing message: {e}")
        response_text = "Oops! Something went wrong. Let me try that again! 🤖"

    # Create TwiML response for WhatsApp
    resp = MessagingResponse()
    resp.message(response_text)

    print(f"🤖 WhatsApp Response: {response_text[:100]}...")
    return str(resp)

@app.route('/webhook/status', methods=['POST'])
def handle_status():
    """Handle delivery status updates (optional)"""
    message_sid = request.form.get('MessageSid')
    message_status = request.form.get('MessageStatus')
    print(f"📊 Message {message_sid} status: {message_status}")
    return '', 200

In [ ]:
def start_server():
    print("🚀 Starting Flask server...")
    print("🌐 Server running on localhost:5000")
    print("📱 Using existing ngrok tunnel")
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
# Usage in Jupyter:
# 1. Make sure you have get_upcoming_events() function available
# 2. Set your API keys
# 3. Run: start_whatsapp_server_simple()
# 4. Configure the ngrok URL in Twilio WhatsApp console
# 5. Join the sandbox and text your WhatsApp!

if __name__ == "__main__":
    start_server()

🚀 Starting Flask server...
🌐 Server running on localhost:5000
📱 Using existing ngrok tunnel
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.68.54:5000
Press CTRL+C to quit
